In [1]:
!git clone "https://github.com/convman/Multimodal-MOSEI"

fatal: destination path 'Multimodal-MOSEI' already exists and is not an empty directory.


In [3]:
cd Multimodal-MOSEI/data/

/content/Multimodal-MOSEI/data


In [0]:
!chmod +x mosei.sh
!./mosei.sh

In [4]:
ls

data/                weights_cnn_happy.h5    weights_rnn_sad.h5
labels/              weights_rnn_angry.h5    y_test_onehot.csv
modify_labels.ipynb  weights_rnn_disgust.h5  y_train_onehot.csv
MOSEI_dataset.ipynb  weights_rnn_fear.h5     y_valid_onehot.csv
mosei.sh*            weights_rnn.h5
weights_cnn.h5       weights_rnn_happy.h5


In [5]:
cd labels

/content/Multimodal-MOSEI/data/labels


In [0]:
import numpy as np
import h5py
import pandas as pd

In [0]:
Test_labels_surprise = pd.read_csv("mosi2uni_Test_labels_surprise.csv",header=None)
Test_labels_angry = pd.read_csv("mosi2uni_Test_labels_angry.csv",header=None)
Test_labels_disgust = pd.read_csv("mosi2uni_Test_labels_disgust.csv",header=None)
Test_labels_fear = pd.read_csv("mosi2uni_Test_labels_fear.csv",header=None)
Test_labels_happy = pd.read_csv("mosi2uni_Test_labels_happy.csv",header=None)
Test_labels_sad = pd.read_csv("mosi2uni_Test_labels_sad.csv",header=None)

In [0]:
Train_labels_surprise = pd.read_csv("mosi2uni_Train_labels_surprise.csv",header=None)
Train_labels_angry = pd.read_csv("mosi2uni_Train_labels_angry.csv",header=None)
Train_labels_disgust = pd.read_csv("mosi2uni_Train_labels_disgust.csv",header=None)
Train_labels_fear = pd.read_csv("mosi2uni_Train_labels_fear.csv",header=None)
Train_labels_happy = pd.read_csv("mosi2uni_Train_labels_happy.csv",header=None)
Train_labels_sad = pd.read_csv("mosi2uni_Train_labels_sad.csv",header=None)

In [10]:
Test_labels_happy.shape

(4832, 1)

In [11]:
print(Test_labels_happy.shape)
print(Train_labels_happy.shape)

(4832, 1)
(15290, 1)


In [13]:
text_train_emb = h5py.File("../data/text_train_emb.h5","r")
print(list(text_train_emb.keys()))
text_train_emb = np.array(text_train_emb.get('d1'))
# text_train_emb = text_train_emb[0:15289]
# text_train_emb = text_train_emb[1:]
text_train_emb.shape

['d1']


(15290, 20, 300)

In [14]:
text_test_emb = h5py.File("../data/text_test_emb.h5","r")
print(list(text_test_emb.keys()))
text_test_emb = np.array(text_test_emb.get('d1'))
# text_test_emb= text_test_emb[0:4831]
# text_test_emb= text_test_emb[1:]
text_test_emb.shape

['d1']


(4832, 20, 300)

In [0]:
import keras
from keras.models import load_model
from keras.models import Model,Sequential,Model
from keras.layers import LSTM,Dense,Input
from keras import callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [0]:
i = Input(shape=(20,300))
x = LSTM(units=256,return_sequences=True)(i)
x = LSTM(units=256,return_sequences=True)(x)
x = LSTM(units=128,return_sequences=True)(x)
x = LSTM(units=128,return_sequences=True)(x)
x = LSTM(units=64,return_sequences=True)(x)
x = LSTM(units=64,return_sequences=True)(x)
x = LSTM(units=32,return_sequences=True)(x)
x = LSTM(units=32,return_sequences=False)(x)
x = Dense(units=32,activation='relu')(x)
x = Dense(units=32,activation='relu')(x)
o = Dense(1,activation='sigmoid')(x)

In [0]:
model_rnn_happy = Model(inputs=i,outputs=o)
model_rnn_angry = Model(inputs=i,outputs=o)
model_rnn_fear = Model(inputs=i,outputs=o)
model_rnn_surprise = Model(inputs=i,outputs=o)
model_rnn_disgust = Model(inputs=i,outputs=o)
model_rnn_sad = Model(inputs=i,outputs=o)

In [40]:
model_rnn_happy.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 20, 300)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 20, 256)           570368    
_________________________________________________________________
lstm_27 (LSTM)               (None, 20, 256)           525312    
_________________________________________________________________
lstm_28 (LSTM)               (None, 20, 128)           197120    
_________________________________________________________________
lstm_29 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
lstm_30 (LSTM)               (None, 20, 64)            49408     
_________________________________________________________________
lstm_31 (LSTM)               (None, 20, 64)            33024     
__________

In [41]:
ls

data/    modify_labels.ipynb  mosei.sh*          y_train_onehot.csv
labels/  MOSEI_dataset.ipynb  y_test_onehot.csv  y_valid_onehot.csv


In [42]:
model_rnn_happy.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_happy.h5",monitor='val_loss',verbose=1)
history_rnn_happy = model_rnn_happy.fit(text_train_emb,Train_labels_happy,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_happy.evaluate(text_test_emb,Test_labels_happy)
print("Saving the model=========================================================")
saved_rnn_happy = load_model('weights_rnn_happy.h5')
print("Evaluating saved model===================================================")
saved_rnn_happy.evaluate(text_test_emb,Test_labels_happy)

W0620 08:13:05.425816 140375192561536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 61s 4ms/step - loss: 0.6807 - acc: 0.5731 - val_loss: 0.6807 - val_acc: 0.5742

Epoch 00001: saving model to weights_rnn_happy.h5
Epoch 2/20
13761/13761 [==============================] - 52s 4ms/step - loss: 0.6693 - acc: 0.6021 - val_loss: 0.6614 - val_acc: 0.6154

Epoch 00002: saving model to weights_rnn_happy.h5
Epoch 3/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.6518 - acc: 0.6254 - val_loss: 0.6511 - val_acc: 0.6279

Epoch 00003: saving model to weights_rnn_happy.h5
Epoch 4/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.6431 - acc: 0.6399 - val_loss: 0.6496 - val_acc: 0.6187

Epoch 00004: saving model to weights_rnn_happy.h5
Epoch 5/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.6332 - acc: 0.6502 - val_loss: 0.6452 - val_acc: 0.6318

Epoch 00005: saving model to weights_rnn_happy.h5
Epoch 6/20


[0.7062204468329221, 0.6148592715231788]

In [43]:
model_rnn_angry.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_angry.h5",monitor='val_loss',verbose=1)
history_rnn_angry = model_rnn_angry.fit(text_train_emb,Train_labels_angry,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_angry.evaluate(text_test_emb,Test_labels_angry)
print("Saving the model=========================================================")
saved_rnn_angry = load_model('weights_rnn_angry.h5')
print("Evaluating saved model===================================================")
saved_rnn_angry.evaluate(text_test_emb,Test_labels_angry)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 58s 4ms/step - loss: 0.5261 - acc: 0.7635 - val_loss: 0.4550 - val_acc: 0.8234

Epoch 00001: saving model to weights_rnn_angry.h5
Epoch 2/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.4995 - acc: 0.7700 - val_loss: 0.4584 - val_acc: 0.8169

Epoch 00002: saving model to weights_rnn_angry.h5
Epoch 3/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.4779 - acc: 0.7774 - val_loss: 0.4572 - val_acc: 0.8136

Epoch 00003: saving model to weights_rnn_angry.h5
Epoch 4/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.4410 - acc: 0.7984 - val_loss: 0.4840 - val_acc: 0.7835

Epoch 00004: saving model to weights_rnn_angry.h5
Epoch 5/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.3889 - acc: 0.8250 - val_loss: 0.5421 - val_acc: 0.7698

Epoch 00005: saving model to weights_rnn_angry.h5
Epoch 6/20


[0.5514814281799146, 0.7754552980132451]

In [44]:
model_rnn_disgust.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_disgust.h5",monitor='val_loss',verbose=1)
history_rnn_disgust = model_rnn_disgust.fit(text_train_emb,Train_labels_disgust,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_disgust.evaluate(text_test_emb,Test_labels_disgust)
print("Saving the model=========================================================")
saved_rnn_disgust = load_model('weights_rnn_disgust.h5')
print("Evaluating saved model===================================================")
saved_rnn_disgust.evaluate(text_test_emb,Test_labels_disgust)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 61s 4ms/step - loss: 0.3827 - acc: 0.8262 - val_loss: 0.3699 - val_acc: 0.8502

Epoch 00001: saving model to weights_rnn_disgust.h5
Epoch 2/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.3370 - acc: 0.8473 - val_loss: 0.3934 - val_acc: 0.8143

Epoch 00002: saving model to weights_rnn_disgust.h5
Epoch 3/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.2871 - acc: 0.8727 - val_loss: 0.4279 - val_acc: 0.8201

Epoch 00003: saving model to weights_rnn_disgust.h5
Epoch 4/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.2244 - acc: 0.9100 - val_loss: 0.4989 - val_acc: 0.7992

Epoch 00004: saving model to weights_rnn_disgust.h5
Epoch 5/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.1774 - acc: 0.9318 - val_loss: 0.5349 - val_acc: 0.7979

Epoch 00005: saving model to weights_rnn_disgust.h5
E

[0.6969439076222727, 0.7781456953642384]

In [45]:
model_rnn_fear.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_fear.h5",monitor='val_loss',verbose=1)
history_rnn_fear = model_rnn_fear.fit(text_train_emb,Train_labels_fear,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_fear.evaluate(text_test_emb,Test_labels_fear)
print("Saving the model=========================================================")
saved_rnn_fear = load_model('weights_rnn_fear.h5')
print("Evaluating saved model===================================================")
saved_rnn_fear.evaluate(text_test_emb,Test_labels_fear)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 63s 5ms/step - loss: 0.2957 - acc: 0.9113 - val_loss: 0.2971 - val_acc: 0.9111

Epoch 00001: saving model to weights_rnn_fear.h5
Epoch 2/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.2767 - acc: 0.9140 - val_loss: 0.2971 - val_acc: 0.9111

Epoch 00002: saving model to weights_rnn_fear.h5
Epoch 3/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.2506 - acc: 0.9148 - val_loss: 0.3198 - val_acc: 0.9065

Epoch 00003: saving model to weights_rnn_fear.h5
Epoch 4/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.1990 - acc: 0.9292 - val_loss: 0.3604 - val_acc: 0.9012

Epoch 00004: saving model to weights_rnn_fear.h5
Epoch 5/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.1430 - acc: 0.9498 - val_loss: 0.3652 - val_acc: 0.8901

Epoch 00005: saving model to weights_rnn_fear.h5
Epoch 6/20
13761

[0.31313243837694066, 0.9045943708609272]

In [46]:
model_rnn_sad.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_sad.h5",monitor='val_loss',verbose=1)
history_rnn_sad = model_rnn_sad.fit(text_train_emb,Train_labels_sad,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_sad.evaluate(text_test_emb,Test_labels_sad)
print("Saving the model=========================================================")
saved_rnn_sad = load_model('weights_rnn_sad.h5')
print("Evaluating saved model===================================================")
saved_rnn_sad.evaluate(text_test_emb,Test_labels_sad)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 64s 5ms/step - loss: 0.5647 - acc: 0.7384 - val_loss: 0.5282 - val_acc: 0.7619

Epoch 00001: saving model to weights_rnn_sad.h5
Epoch 2/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.5052 - acc: 0.7543 - val_loss: 0.5391 - val_acc: 0.7449

Epoch 00002: saving model to weights_rnn_sad.h5
Epoch 3/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.4137 - acc: 0.8062 - val_loss: 0.6021 - val_acc: 0.7279

Epoch 00003: saving model to weights_rnn_sad.h5
Epoch 4/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.2965 - acc: 0.8704 - val_loss: 0.7356 - val_acc: 0.6808

Epoch 00004: saving model to weights_rnn_sad.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.2047 - acc: 0.9140 - val_loss: 0.8761 - val_acc: 0.6854

Epoch 00005: saving model to weights_rnn_sad.h5
Epoch 6/20
13761/1376

[1.1716919887243518, 0.6612168874172185]

In [47]:
model_rnn_surprise.compile('adam','binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001,verbose=1)
mcp = ModelCheckpoint("weights_rnn_surprise.h5",monitor='val_loss',verbose=1)
history_rnn_surprise = model_rnn_surprise.fit(text_train_emb,Train_labels_surprise,64,20,validation_split=0.1,callbacks=[es, mcp])
print("Evaluating the model=====================================================")
model_rnn_surprise.evaluate(text_test_emb,Test_labels_surprise)
print("Saving the model=========================================================")
saved_rnn_surprise = load_model('weights_rnn_surprise.h5')
print("Evaluating saved model===================================================")
saved_rnn_surprise.evaluate(text_test_emb,Test_labels_surprise)

Train on 13761 samples, validate on 1529 samples
Epoch 1/20
13761/13761 [==============================] - 67s 5ms/step - loss: 0.3499 - acc: 0.8887 - val_loss: 0.2966 - val_acc: 0.9091

Epoch 00001: saving model to weights_rnn_surprise.h5
Epoch 2/20
13761/13761 [==============================] - 51s 4ms/step - loss: 0.3151 - acc: 0.8966 - val_loss: 0.2959 - val_acc: 0.9091

Epoch 00002: saving model to weights_rnn_surprise.h5
Epoch 3/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.2852 - acc: 0.8977 - val_loss: 0.3130 - val_acc: 0.9078

Epoch 00003: saving model to weights_rnn_surprise.h5
Epoch 4/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.2191 - acc: 0.9182 - val_loss: 0.3428 - val_acc: 0.8960

Epoch 00004: saving model to weights_rnn_surprise.h5
Epoch 5/20
13761/13761 [==============================] - 50s 4ms/step - loss: 0.1428 - acc: 0.9462 - val_loss: 0.4047 - val_acc: 0.8672

Epoch 00005: saving model to weights_rnn_surprise

[0.6632042229776919, 0.8727235099337748]

In [48]:
ls

data/                weights_rnn_angry.h5    weights_rnn_surprise.h5
labels/              weights_rnn_disgust.h5  y_test_onehot.csv
modify_labels.ipynb  weights_rnn_fear.h5     y_train_onehot.csv
MOSEI_dataset.ipynb  weights_rnn_happy.h5    y_valid_onehot.csv
mosei.sh*            weights_rnn_sad.h5


In [0]:
from google.colab import files
files.download('weights_rnn_angry.h5')
files.download('weights_rnn_disgust.h5')
files.download('weights_rnn_fear.h5')
files.download('weights_rnn_happy.h5')
files.download('weights_rnn_sad.h5')
files.download('weights_rnn_surprise.h5')

In [0]:
from keras.layers import Conv1D,MaxPool1D,Flatten

In [0]:
model_cnn = Sequential()
model_cnn.add(Conv1D(32,3,padding='same',activation = 'relu',input_shape=(20,300)))
model_cnn.add(Conv1D(32,3,padding='same',activation = 'relu'))
model_cnn.add(MaxPool1D(2))
model_cnn.add(Conv1D(64,3,padding='same',activation = 'relu'))
model_cnn.add(Conv1D(64,3,padding='same',activation = 'relu'))
model_cnn.add(MaxPool1D(2))
model_cnn.add(Conv1D(128,3,padding='same',activation = 'relu'))
model_cnn.add(Conv1D(128,3,padding='same',activation = 'relu'))
model_cnn.add(MaxPool1D(2))
model_cnn.add(Flatten())
model_cnn.add(Dense(1024,activation='relu'))
model_cnn.add(Dense(512,activation='relu'))
model_cnn.add(Dense(128,activation='relu'))
model_cnn.add(Dense(1,activation='sigmoid'))

In [82]:
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 20, 32)            28832     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 20, 32)            3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 10, 32)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 10, 64)            6208      
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 10, 64)            12352     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 5, 64)             0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 5, 128)            24704     
__________

In [0]:
model_cnn.compile('adam','binary_crossentropy',['accuracy'])

In [0]:
es = EarlyStopping(monitor='val_loss',mode='min' ,patience=5, min_delta=0.0001)
mcp = ModelCheckpoint("weights_cnn_happy.h5",monitor='val_loss',verbose=1)

In [86]:
history_cnn = model_cnn.fit(text_train_emb,Train_labels_happy,64,20,validation_split=0.1,callbacks=[es,mcp])

Train on 13760 samples, validate on 1529 samples
Epoch 1/20
13760/13760 [==============================] - 12s 876us/step - loss: 0.6675 - acc: 0.5930 - val_loss: 0.6526 - val_acc: 0.6050

Epoch 00001: saving model to weights_cnn_happy.h5
Epoch 2/20
13760/13760 [==============================] - 2s 150us/step - loss: 0.6417 - acc: 0.6383 - val_loss: 0.6492 - val_acc: 0.6135

Epoch 00002: saving model to weights_cnn_happy.h5
Epoch 3/20
13760/13760 [==============================] - 2s 145us/step - loss: 0.6274 - acc: 0.6587 - val_loss: 0.6507 - val_acc: 0.6167

Epoch 00003: saving model to weights_cnn_happy.h5
Epoch 4/20
13760/13760 [==============================] - 2s 145us/step - loss: 0.5986 - acc: 0.6882 - val_loss: 0.6532 - val_acc: 0.6409

Epoch 00004: saving model to weights_cnn_happy.h5
Epoch 5/20
13760/13760 [==============================] - 2s 144us/step - loss: 0.5598 - acc: 0.7174 - val_loss: 0.6872 - val_acc: 0.6128

Epoch 00005: saving model to weights_cnn_happy.h5
Epoch

In [88]:
model_cnn.evaluate(text_test_emb,Test_labels_happy)

4831/4831 [==============================] - 1s 114us/step


[0.7884256262829574, 0.5864210306944203]

In [89]:
from keras.models import load_model
saved_cnn = load_model('weights_cnn_happy.h5')
saved_cnn.evaluate(text_test_emb,Test_labels_happy)

4831/4831 [==============================] - 4s 774us/step


[0.7884256262829574, 0.5864210306944203]